# **Install and Import Libraries**

> ##### **create .streamlit/secrets.toml file inside notebooks folder with following lines:**

> ###### **OPENAI_API_KEY = "<api_key>"**


In [ ]:
%pip install ../.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

import sys
from data_pipeline import *

In [ ]:
import os
os.getenv('AGENDA_EXTRACTION_PROMPT_PATH')

# **1. Scrape Website**

In [ ]:
df = scrape_website()

In [ ]:
df

# **2. Download all PDFs from links**

In [ ]:
df = download_documents()

In [ ]:
df

# **4. Extract HTML from PDFs**

In [ ]:
convert_pdf_to_html()

# **4. Extract Meeting Metadata from PDF with LLM**

In [ ]:
df = extract_meeting_metadata()

In [ ]:
df

# **5. Extract Proposals and Decisions**

## still not working as expected

In [ ]:
df = extract_meeting_agenda()